In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

In [ ]:
from tda.experiments.mahalanobis.mahalanobis_binary import Config, run_experiment, get_feature_datasets, evaluate_detector, \
    compute_means_and_sigmas_inv, get_config, logger
from tda.models.architectures import mnist_lenet, get_architecture, mnist_mlp
from tda.models.datasets import Dataset
from tda.models import get_deep_model

In [ ]:
config = Config(
    # Number of epochs for the model
    epochs = 50,
    # Dataset we consider (MNIST, SVHN)
    dataset = "MNIST",
    # Name of the architecture
    architecture = mnist_lenet.name,
    # Size of the dataset used for the experiment
    dataset_size = 1000,
    # Type of attack (FGSM, BIM, CW)
    attack_type = "FGSM",
    # Epsilon for the preprocessing step (see the paper)
    preproc_epsilon = 0.0,
    # Noise to consider for the noisy samples
    noise = 0.0,
    # Number of sample per class to estimate mu_class and sigma_class
    number_of_samples_for_mu_sigma = 100,
    # Should we filter out non successful_adversaries
    successful_adv = 0
)

In [ ]:
dataset = Dataset(name=config.dataset)

architecture = get_deep_model(
    num_epochs=config.epochs,
    dataset=dataset,
    architecture=get_architecture(config.architecture),
    train_noise=0.0
)

mean_per_class, sigma_per_class_inv = compute_means_and_sigmas_inv(
    config=config,
    dataset=dataset,
    architecture=architecture
)

In [ ]:
fig, axs = plt.subplots(len(sigma_per_class_inv)//2+1, 2)
i = 0

for layer_idx in sigma_per_class_inv:
    px = i // 2
    py = i % 2
    axs[px][py].imshow(sigma_per_class_inv[layer_idx])
    i+=1

In [ ]:
datasets = dict()
for epsilon in [0.01, 0.025, 0.05, 0.1, 0.4, 1.0]:
    ds_train, ds_test = get_feature_datasets(
        config=config,
        epsilon=epsilon,
        dataset=dataset,
        architecture=architecture,
        mean_per_class=mean_per_class,
        sigma_per_class_inv=sigma_per_class_inv
    )
    
    datasets[epsilon] = (ds_train, ds_test)

In [ ]:
#datasets = dict()
#for num_iter in [1, 5, 10, 20, 30, 50]:
#    ds_train, ds_test = get_feature_datasets(
#        config=config,
#        epsilon=1.0,
#        num_iter=num_iter,
#        dataset=dataset,
#        architecture=architecture,
#        mean_per_class=mean_per_class,
#        sigma_per_class_inv=sigma_per_class_inv
#    )
#    
#    datasets[num_iter] = (ds_train, ds_test)

In [ ]:
datasets[1.0][0]

## Supervised Learning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np

scaler = StandardScaler()

aucs = dict()
coefs = dict() 

for key in datasets:
    
    ds_train, ds_test = datasets[key]
    
    typ_detector = "LR"
    
    if typ_detector == "LR":
        detector = LogisticRegression(
        fit_intercept=True,
        verbose=0,
        tol=1e-9,
        max_iter=100000,
        solver='lbfgs'
    )
    elif typ_detector == "SVM":
        detector = SVC(
            tol=1e-5,
            C=1.0,
            kernel="rbf",
            verbose=False,
            max_iter=10000,
            gamma='scale'
        )
    else:
        raise NotImplementedError(f"Unknown detector method {typ_detector}")
        
    ds_train_scaled = scaler.fit_transform(ds_train.iloc[:, :-3])

    detector.fit(X=ds_train_scaled, y=ds_train.iloc[:, -3])
    #coef = list(detector.coef_.flatten())
    #logger.info(f"Coefs of detector {coef}")
    #logger.info(f"Intercept of detector {detector.intercept_}")
    
    ds_test_scaled = scaler.transform(ds_train.iloc[:, :-3])
    
    train_predictions = detector.predict_proba(X=ds_train_scaled)[:, 1]
    
    test_predictions = detector.predict_proba(X=ds_test_scaled)[:, 1]
    logger.info(f"Average prediction {np.mean(test_predictions)}")
    auc = roc_auc_score(y_true=ds_test.iloc[:, -3], y_score=test_predictions)
    
    logger.info(f"AUC for {key} => {auc}")

    aucs[key] = auc
    #coefs[key] = coef
    
logger.info(f"All AUCS are {aucs}")

In [ ]:
linfs = dict()
l2s = dict()

for key in datasets:
    linf_mean_train = datasets[key][0].query("label > 0").linf_norm.mean()
    l2_mean_train = datasets[key][0].query("label > 0").l2_norm.mean()
    
    linfs[key] = linf_mean_train
    l2s[key] = l2_mean_train

In [ ]:
keys = sorted(aucs.keys())
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epsilon')
ax1.set_ylabel('AUC & l_inf norm')
#plt.xscale('log')
ax1.plot(
    keys, [aucs[k] for k in keys], label="Mahalanobis"
)
ax1.plot(
    keys, [linfs[k] for k in keys], label="l_inf"
)

ax2 = ax1.twinx()
ax2.set_ylabel("l_2 norm")

ax2.plot(
    keys, [l2s[k] for k in keys], label="l_2", color="green"
)

ax1.legend()
ax2.legend()

## Unsupervised Learning

In [ ]:
from sklearn.svm import OneClassSVM

scaler = StandardScaler()

aucs = dict()
coefs = dict() 

for key in datasets:
    
    ds_train, ds_test = datasets[key]
    
    detector = OneClassSVM(
        tol=1e-5,
        kernel='rbf',
        nu=1e-4
    )
        
    ds_train_scaled = scaler.fit_transform(ds_train.iloc[:, :-3])

    detector.fit(X=ds_train_scaled)
    #coef = list(detector.coef_.flatten())
    #logger.info(f"Coefs of detector {coef}")
    #logger.info(f"Intercept of detector {detector.intercept_}")
    
    ds_test_scaled = scaler.transform(ds_train.iloc[:, :-3])
    
    train_predictions = detector.decision_function(X=ds_train_scaled)
    
    test_predictions = detector.decision_function(X=ds_test_scaled)
    logger.info(f"Average prediction {np.mean(test_predictions)}")
    auc = roc_auc_score(y_true=ds_test.iloc[:, -3], y_score=test_predictions)
    
    logger.info(f"AUC for {key} => {auc}")

    aucs[key] = auc
    #coefs[key] = coef
    
logger.info(f"All AUCS are {aucs}")

In [ ]:
keys = sorted(aucs.keys())
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

fig, ax1 = plt.subplots()

ax1.set_xlabel('epsilon')
ax1.set_ylabel('AUC & l_inf norm')
#plt.xscale('log')
ax1.plot(
    keys, [aucs[k] for k in keys], label="Mahalanobis"
)
ax1.plot(
    keys, [linfs[k] for k in keys], label="l_inf"
)

ax2 = ax1.twinx()
ax2.set_ylabel("l_2 norm")

ax2.plot(
    keys, [l2s[k] for k in keys], label="l_2", color="green"
)

ax1.legend()
ax2.legend()